# Exploratory Data Analysis Notebook
This notebook will preprocess and leverage NLP models on the unstructured data to turn it into a usable feature space for modeling Tucker Carlson's body of work

In [1]:
#Imports cell

#Import basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import csv
import nltk
from nltk import word_tokenize
from nltk import FreqDist
import re
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
# import the tucker document data either as a CSV or a pickle

#Read out from CSV
tucker_docs = pd.read_csv('data/tucker_docs.csv', encoding='UTF8', header = None).T

#Read from pickle
#tucker_docs = pd.read_pickle('data/tucker_pickle')

In [14]:
tucker_docs.head()

,0
0,Fox News host gives his take on pro-abortion ...
1,Fox News host reflects on the left's respons...
2,Fox News host gives his take on how Americans...
3,Fox News host gives his take on the Supreme C...
4,Fox News host gives his take on the real moti...


## Implementing the Bradley-Haderthauer Test
Compare two topic distributions: IF BH-score is < 0.2, then a Twitterer can be confidently classified as a Tuckerbot. This person is a lower life form and unable to contribute, in good faith, to the deep state media platform of choice, Twitter.

In [15]:
#make custom stops words to remove first 100 words? remove intro to episode 
#remove words in all caps 

## remove words in all caps

In [16]:


tucker_doc = tucker_docs.iloc[0,0]
#pattern to delete words in all caps
#pattern = "(([a-zA-Z]+(?:'[a-z]+)?))"
t_d = re.sub(r'\b[A-Z]+\b', '', tucker_doc)
pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
tokenized_doc = nltk.regexp_tokenize(t_d, pattern)

#lowercase all words
tokenized_doc = [word.lower() for word in tokenized_doc]
tokenized_doc

#Freqdist
td_freqdist = FreqDist(tokenized_doc)
td_freqdist.most_common(75)

#Stop words
stopwords_list = stopwords.words('english')
stop_tokenized_doc = [word for word in tokenized_doc if word not in stopwords_list]

#Stopped freqdist 
stop_td_freqdist = FreqDist(stop_tokenized_doc)
stop_td_freqdist.most_common(75)

#lemmatize
#to lem 
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

#more lemmatizin
#wnl.lemmatize(wn1.lemmatize(word) for word in stop_tokenized_doc)
tokens = [wnl.lemmatize(word) for word in stop_tokenized_doc]
tokens

#lemmatizer = WordNetLemmatizer()
#def lemmatize_words(text):
   # return " ".join([lemmatizer.lemmatize(word) for word in text.split()])
#stop_tokenized_doc = stop_tokenized_doc.apply(lambda text: lemmatize_words(stop_tokenized_doc))

['fox',
 'news',
 'host',
 'give',
 'take',
 'pro',
 'abortion',
 'protester',
 'targeting',
 'supreme',
 'court',
 'justice',
 'possible',
 'overturn',
 'roe',
 'v',
 'wade',
 'tucker',
 'carlson',
 'tonight',
 'pretty',
 'hard',
 'argue',
 'people',
 'passive',
 'aggressive',
 'may',
 'tried',
 'angry',
 'scream',
 'stop',
 'violent',
 'snarl',
 'punch',
 'face',
 'passive',
 'aggressive',
 'people',
 'intent',
 'dominating',
 "they're",
 'dishonest',
 'admit',
 'honorable',
 'style',
 'attack',
 'effective',
 'mostly',
 'bewildering',
 'democratic',
 'party',
 'practice',
 'democrat',
 'never',
 'meet',
 'open',
 'field',
 'battle',
 'instead',
 'sneak',
 'behind',
 'knock',
 'unconscious',
 'bag',
 'sanctimony',
 'party',
 'weak',
 'men',
 'angry',
 'woman',
 'passive',
 'aggression',
 'mode',
 'communication',
 'ever',
 'seen',
 'one',
 'jen',
 "psaki's",
 'press',
 'conference',
 'know',
 'exactly',
 "we're",
 'talking',
 'watched',
 'one',
 'yesterday',
 'fact',
 'last',
 'peter

### Preprocess function to do all steps above at once

In [17]:
def preprocessing(text):

    #Step 0
    #This is Praveen's code but it's not PEP-8 friendly so you should fix that for next cohort ty
    text = text.lower()
    text = re.sub(r"@[a-z0-9_]+|#[a-z0-9_]+@[A-Z0-9_]+|#[A-Z0-9_]+|http\S+", "", text).strip().replace("\r", "").replace("\n", "").replace("\t", "")
   
    #step 1: delete all caps words
    t_d = re.sub(r'\b[A-Z]+\b', '', text)

    #step 2: tokenize
    pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
    tokenized_doc = nltk.regexp_tokenize(t_d, pattern)

    #step 3: stop words
    stopwords_list = stopwords.words('english')
    stop_tokenized_doc = [word for word in tokenized_doc if word not in stopwords_list]

    #step 4: lem
    tokens = [wnl.lemmatize(word) for word in stop_tokenized_doc]
    return ' '.join(tokens)

In [18]:
tucker_list = tucker_docs[0].tolist()
new_list = []
for each_doc in tucker_list:
    new_list.append(preprocessing(each_doc))

In [19]:
feature_names = new_list[0][0].get_feature_names()
not_so_sparse_not_so_spicy = pd.DataFrame(new_list[0][1].toarray(), columns = feature_names)

AttributeError: 'str' object has no attribute 'get_feature_names'

In [82]:
not_so_sparse_not_so_spicy.sort_values(by = ['address'])

,able,abortion,abortionist,absurd,according,across,activist,actual,actually,address,...,worse,would,write,wrote,ya,yeah,year,yesterday,york,young
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
782,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
781,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
780,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
779,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
392,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
384,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
101,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [118]:
vectorize= CountVectorizer()

In [120]:
vect = vectorize.fit_transform(new_list[0])

In [121]:
vect

<1169x660 sparse matrix of type '<class 'numpy.int64'>'
	with 1174 stored elements in Compressed Sparse Row format>

In [123]:
from sklearn.cluster import KMeans
kmeans = KMeans()

## Preprocess the twitter data

In [20]:
tweet_df = pd.read_csv('data/unique_tweets_list.csv').drop(columns = ['Unnamed: 0'])
#tweet_df['tweet_history'][0]

In [21]:
import ast

len(ast.literal_eval(tweet_df.iloc[750,:][0]))

100

In [22]:
def string_to_list(s):
    try:
        l = ast.literal_eval(s)
    except:
        l = None
    return l

In [23]:
tweet_df['tweet_history'] = tweet_df['tweet_history'].apply(lambda x: string_to_list(x))

In [24]:
tweet_df = tweet_df.dropna()

In [30]:
def prepare_tweets(tweet_row):
    tweet_list = []
    for n in range(len(tweet_row)):
        tweet_list.append(preprocessing(tweet_row[n]))

    cleaned_tweets = []
    for tweet in tweet_list:
        if len(tweet) > 20:
            cleaned_tweets.append(tweet)
    return cleaned_tweets

In [31]:
tweet_df['cleaned_tweets'] = tweet_df['tweet_history'].apply(lambda x: prepare_tweets(x))

In [38]:
tweet_df['cleaned_tweets']

['legislative genius nancy need go away take diane done useless corporate democrat',
 'imagine female journos stay twitter musk charge twitter journalism lazy journalism anyway leave force orgs leave well msn departs twitter done simple',
 'poised lose congress party ban book amp reproductive right amp record end s mc think colossally effed pull public begging fight paralyzed',
 "good riddance let's watch replace withsomeone even worse",
 "hell's party standing firmly resolutely many u asking since turn performative b kabuki theater win fight would thunk",
 'ana kasparian young turk absolutely killing last day tired feckless corp might want tune party beholden corp donor interest way priority list',
 'stop letting asshats play u performative fundraising nonsense find way vote people give damn u fight u',
 'clear response elected dems scotus decision vote senate bill chance passing filibuster confront say get everyone record everyone know already',
 'protected abortion right vigorously 

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer

tweet_vectorizer = TfidfVectorizer()
tweet_df['vectorized'] = tweet_df['cleaned_tweets'].apply(lambda x: tweet_vectorizer.(x))